In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
df=pd.read_csv("/kaggle/input/waterneeded/TARP.csv")
df

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(how="any",axis=0)
df

In [ ]:
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(df["Status"])
Y

In [ ]:
X=df.drop(columns=["Status"])

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [8]:
X_train

,Soil Moisture,Temperature,Soil Humidity,Time,Air temperature (C),Wind speed (Km/h),Air humidity (%),Wind gust (Km/h),Pressure (KPa),ph,rainfall,N,P,K
1656,67,43,21,69,29.59,12.37,3.03,69.95,101.38,6.625539,102.944161,17.0,16.0,14.0
752,39,31,20,49,33.38,14.92,1.66,78.88,101.16,7.143943,69.408782,37.0,79.0,19.0
892,76,42,39,86,18.94,7.02,20.29,17.91,101.30,7.288057,45.208411,7.0,73.0,25.0
1041,80,20,42,1,31.54,13.68,4.33,82.76,101.14,6.012697,116.553145,101.0,70.0,48.0
1179,42,36,56,6,19.31,5.32,23.17,12.47,101.44,6.279134,97.790725,0.0,17.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,7,0,69,41,31.40,12.43,2.04,72.43,101.34,7.817846,112.983436,10.0,5.0,5.0
1095,7,39,60,63,26.57,6.26,6.98,13.41,101.18,6.387431,90.812505,108.0,94.0,47.0
1130,6,36,55,53,23.45,5.72,16.82,21.43,101.40,6.475449,100.258567,11.0,36.0,31.0
1294,65,18,62,86,31.08,14.07,5.76,75.00,101.65,6.361141,71.400430,11.0,124.0,204.0


In [20]:
for i in [RandomForestClassifier(),LogisticRegression(),GradientBoostingClassifier(),AdaBoostClassifier(),KNeighborsClassifier(),BernoulliNB(),SVC()]:
    random_classif=i
    random_classif.fit(X_train,Y_train)
    
    print(f"The Accuracy of {i}:",random_classif.score(X_test,Y_test))

The Accuracy of RandomForestClassifier(): 0.8863636363636364
The Accuracy of LogisticRegression(): 0.7159090909090909


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The Accuracy of GradientBoostingClassifier(): 0.9272727272727272
The Accuracy of AdaBoostClassifier(): 0.85
The Accuracy of KNeighborsClassifier(): 0.775
The Accuracy of BernoulliNB(): 0.5659090909090909
The Accuracy of SVC(): 0.8136363636363636


# # Quantum Deep learning

In [ ]:
!pip install cirq

In [32]:
import cirq
import tensorflow as tf
import pandas as pd
from keras.layers import Layer,Dense,Dropout,BatchNormalization
from keras.models import Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2

In [34]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(14)]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        q=i%len(qubit)
        circuit.append(cirq.H(qubit[q]))
        circuit.append(cirq.rx(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.X(qubit[q]))
        circuit.append(cirq.rz(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.Z(qubit[q]))
        circuit.append(cirq.ry(value*2*np.pi).on(qubit[q]))
        circuit.append(cirq.Y(qubit[q]))
        if (i+2)%len(qubit)< len(qubit):
            circuit.append(cirq.CNOT(qubit[q],qubit[(i+1)%len(qubit)]))
            circuit.append(cirq.CX(qubit[(i+1)%len(qubit)],qubit[(i+2)%len(qubit)]))
            circuit.append(cirq.measure(qubit[q],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    simulator=cirq.Simulator()
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        outputs=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        outputs.set_shape(inputs.shape)
        return outputs
model=Sequential([
    QuantumLayer(),
    Dense(32,activation="relu"),
    Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    Dense(1,activation="sigmoid")
])
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])


In [35]:
model.fit(X_train,Y_train,epochs=15,batch_size=32)

Epoch 1/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5215 - loss: 4.1853
Epoch 2/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5383 - loss: 2.9701
Epoch 3/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.4997 - loss: 2.1790
Epoch 4/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.5157 - loss: 1.6623
Epoch 5/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5448 - loss: 1.3174
Epoch 6/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5323 - loss: 1.0977
Epoch 7/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5301 - loss: 0.9532
Epoch 8/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5386 - loss: 0.8594
Epoch 9/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.5398 - loss: 0.7984
Epoch 10/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.5465 - loss: 0.7588
Epoch 11/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.5230 - loss: 0.7368
Epoch 12/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.5294 - lo

In [36]:
model1=Sequential([
    Dense(32,activation="relu"),

    QuantumLayer(),
    # Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    # Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    Dense(1,activation="sigmoid")
])
model1.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])
model1.fit(X_train,Y_train,epochs=15,batch_size=32)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


55/55 ━━━━━━━━━━━━━━━━━━━━ 152s 3s/step - accuracy: 0.5045 - loss: 0.7385
Epoch 2/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5028 - loss: 0.7448
Epoch 3/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5196 - loss: 0.7362
Epoch 4/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.4809 - loss: 0.7490
Epoch 5/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.5165 - loss: 0.7351
Epoch 6/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.5257 - loss: 0.7196
Epoch 7/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5199 - loss: 0.7129
Epoch 8/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5350 - loss: 0.7077
Epoch 9/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.5236 - loss: 0.7112
Epoch 10/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.5087 - loss: 0.7122
Epoch 11/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5192 - loss: 0.7048
Epoch 12/15
55/55 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5115 - l

In [40]:
import cirq
import numpy as np
def run_quantum_circuit(data):
    # Generate quantum circuit
    circuit, qubits = quantum_circuit(data)
    
    # Run the circuit on a simulator
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1)
    
    # Extract measurements
    measure0 = []
    for i in range(len(qubits)):
        key = f"qubit_{i}"
        if key in result.measurements:
            measure0.append(result.measurements[key].flatten()[0])
        else:
            raise KeyError(f"Measurement key {key} not found in results.")
    
    return measure0

def run_quantum_circuit(data):
    # Generate quantum circuit
    circuit, qubits = quantum_circuit(data)
    
    # Run the circuit on a simulator
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1)
    
    # Extract measurements
    measure0 = []
    for i in range(len(qubits)):
        key = f"qubit_{i}"
        if key in result.measurements:
            measure0.append(result.measurements[key].flatten()[0])
        else:
            raise KeyError(f"Measurement key {key} not found in results.")
    
    return measure0

class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        outputs=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        outputs.set_shape(inputs.shape)
        return outputs
model2=Sequential([
    Dense(32,activation="relu"),

    QuantumLayer(),
    # Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    # Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    Dense(1,activation="sigmoid")
])
model2.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])
model2.fit(X_train,Y_train,epochs=15,batch_size=32)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node sequential_10_1/quantum_layer_10_1/PyFunc defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py",

In [11]:
import cirq
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Quantum Circuit Definition
def quantum_circuit(data):
    num_qubits = 14
    qubits = [cirq.GridQubit(0, i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Data encoding and processing
    for i, value in enumerate(data):
        q = i % num_qubits
        circuit.append(cirq.rx(value * 2 * np.pi).on(qubits[q]))
        circuit.append(cirq.rz(value * 2 * np.pi).on(qubits[q]))
    for i in range(num_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[(i + 1) % num_qubits]))
    circuit.append([cirq.measure(q, key=f'qubit_{i}') for i, q in enumerate(qubits)])
    return circuit, qubits

# Running the Quantum Circuit
def run_quantum_circuit(data):
    results = []
    simulator = cirq.Simulator()
    for inputs_data in data:
        circuit, qubits = quantum_circuit(inputs_data)
        result = simulator.run(circuit)
        measure0 = [result.measurements.get(f"qubit_{i}", np.array([[0]])).flatten()[0] for i in range(len(qubits))]
        results.append(measure0)
    return np.array(results, dtype=np.float32)

# Custom Quantum Layer
class QuantumLayer(Layer):
    def __init__(self, num_qubits=14):
        super(QuantumLayer, self).__init__()
        self.num_qubits = num_qubits

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        reshaped_inputs = tf.reshape(inputs, (batch_size, self.num_qubits))
        outputs = tf.numpy_function(run_quantum_circuit, [reshaped_inputs], tf.float32)
        outputs.set_shape((None, self.num_qubits))
        return outputs

# Model Definition
model = Sequential([
    Dense(32, activation="relu"),
    QuantumLayer(num_qubits=14),
    Dense(64, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(1, activation="sigmoid")
])

# Compilation and Training
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train[:, :14], Y_train, epochs=15, batch_size=32)  # Ensure input matches qubit count


InvalidIndexError: (slice(None, None, None), slice(None, 14, None))

In [ ]:
import cirq
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.models import Sequential

# Quantum Circuit Definition
def quantum_circuit(data):
    num_qubits = len(data)  # Adjust dynamically to match input data size
    qubits = [cirq.GridQubit(0, i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Data encoding and quantum processing
    for i, value in enumerate(data):
        circuit.append(cirq.rx(value * 2 * np.pi).on(qubits[i]))
        circuit.append(cirq.rz(value * 2 * np.pi).on(qubits[i]))

    # Entangling layer
    for i in range(num_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[(i + 1) % num_qubits]))

    # Measurement
    circuit.append([cirq.measure(q, key=f'qubit_{i}') for i, q in enumerate(qubits)])
    return circuit, qubits

# Running the Quantum Circuit
def run_quantum_circuit(data):
    results = []
    simulator = cirq.Simulator()
    for inputs_data in data:
        circuit, qubits = quantum_circuit(inputs_data)
        result = simulator.run(circuit)
        measurements = [
            result.measurements[f'qubit_{i}'][0][0] for i in range(len(qubits))
        ]
        results.append(measurements)
    return np.array(results, dtype=np.float32)

# Custom Quantum Layer
class QuantumLayer(Layer):
    def __init__(self, num_qubits=14, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.num_qubits = num_qubits

    def call(self, inputs):
        # batch_size = tf.shape(inputs)[0]
        # reshaped_inputs = tf.reshape(inputs, (batch_size, self.num_qubits))
        outputs = tf.numpy_function(run_quantum_circuit, [inputs], tf.float32)
        outputs.set_shape((None,self.num_qubits))  # Dynamically infer batch size
        return outputs

# Model Definition
model = Sequential([
    Dense(32, activation="relu"),
    QuantumLayer(num_qubits=14),
    Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(1, activation="sigmoid")
])

# Compilation
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

# Training Input Data (Ensure correct shape)
# X_train: Training data with at least 14 features
# Y_train: Corresponding labels
if X_train.shape[1] < 14:
    raise ValueError("Input data must have at least 14 features for the quantum layer.")

# Model Training
model.fit(X_train.iloc[:, :14], Y_train, epochs=15, batch_size=32)
